## 1.정수 인코딩(Integer Encoding)


컴퓨터는 텍스트보다 숫자를 더 잘 처리할 수 있다. 이를 위해 자연어 처리에서는 텍스트를 숫자로 바꾸는 여러가지 기법들이 있다.

그리고 그러한 기법들을 본격적으로 적용시키기 위한 첫 단계로 각 단어를 고유한 정수에 맵핑(mapping)시키는 전처리 작업이 필요할 때가 있다.

예를 들어 갖고 있는 텍스트에 단어가 5000개가 있다면 5000개의 단어들 각각에 1번부터 5000번까지 단어와 맵핑되는 고유한 정수, 다른 표현으로는 인덱스를 부여한다.

인덱스를 부혀하는 방법은 여러 가지가 있을 수 있는데 랜덤으로 부여하기도 하지만, 보통은 전처리 또는 빈도수가 높은 단어들만 사용하기 위해서 단어에 대한 빈도수를 기준으로 정렬한 뒤에 부여한다.

***
왜 이런 작업이 필요한 지에 대해서는 뒤에서 원-핫 인코딩, 워드 임베딩 챕터 등에서 알아보기로 하고 여기서는 어떤 과정으로 단어에 정수 인덱스를 부여하는지에 대해서만 정리하자.

단어에 정수를 부여하는 방법 중 하나로 단어를 빈도수 순으로 정렬한 단어 집합(vocabulary)을 만들고, 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여하는 방법이 있다. 

이해를 돕기위해 단어의 빈도수가 적당하게 분포되도록 의도적으로 만든 텍스트 데이터를 가지고 실습해보자.

### 1) dictionary 사용하기

In [1]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [2]:
text = "A barber is a person. a barber is good person. a barber is huge person. \
he Knew A Secret! The Secret He Kept is huge secret. Huge secret. \
His barber kept his word. a barber kept his word. His barber kept his secret. \
But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [3]:
# 문장 토큰화
text = sent_tokenize(text)
print(text)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [4]:
# 정제와 단어 토큰화
vocab = {}
sentences = []
stop_words = set(stopwords.words('english'))

for i in text :
    sentence = word_tokenize(i) # 단어 토큰화를 수행
    result = []
    
    for word in sentence :
        word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄인다.
        if word not in stop_words : # 단어 토큰화 된 결과에 대해서 불용어를 제거한다.
            if len(word) > 2 : # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거한다.
                result.append(word)
                
                if word not in vocab :  # vocab에 word가 없을 경우
                    vocab[word] = 0 # value는 0을 입력
                vocab[word] += 1 # value 값을 1씩 더해나감
    sentences.append(result)

print(sentences)
        

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


텍스트를 숫자로 바꾸는 단계라는 것은 본격적으로 자연어 처리 작업에 들어간다는 의미이므로, 단어가 텍스트일때만 할 수 있는 최대한의 전처리를 끝내놓아야 한다.

위의 코드에서는 동일한 단어가 대문자로 표기되었다는 이유로 서로 다른 단어로 카운트되는 일이 없도록 모든 단어를 소문자로 바꾸고, 자연어 처리에서 크게 의미를 갖지 못하는 불용어와 길이가 짧은 단어를 제거하는 방법을 사용했다.

In [5]:
print(vocab)

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


vocab에는 중복을 제거한 단어와 각 단어에 대한 빈도수가 기록되어져 있다.

In [6]:
# 해당 단어의 빈도수 출력
print(vocab['barber'])

8


빈도수가 높은 순서대로 정렬

In [7]:
vocab_sorted = sorted(vocab.items(), key = lambda x :x[1], reverse = True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여한다.

In [8]:
word_to_index = {}

i = 0

for (word, frequency) in vocab_sorted :
    if frequency > 1 : # 빈도 수가 적은 단어는 제외
        i = i+1
        word_to_index[word] = i
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


1의 인덱스를 가진 단어가 가장 빈도수가 높은 단어가 된다.

이러한 작업을 수행하는 동시에 각 단어의 빈도수를 알 경우에만 할 수있는 전처리인 빈도수가 적은 단어를 제외시키는 작업을 한다.(등장 빈도가 낮은 단어는 자연어 처리에서 의미를 가지지 않을 가능성이 높기 때문이다.)

여기서는 빈도수가 1인 단어들을 전부 제외했다.


자연어 처리를 하다보면 빈도수가 가장 높은 n개의 단어만 사용하고 싶은 경우가 많은데, 
현재 빈도수가 높은 순으로 낮은 정수가 부여되어져 있으므로 빈도수 상위 n개의 단어들만 사용하면 된다.

In [9]:
# 상위 5개의 단어들만 사용

vocab_size = 5
# 인덱스가 5보다 높은 단어들 추출 
words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1]

# 추출한 단어들을 삭제
for w in words_frequency :
    del word_to_index[w]
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


이제 word_to_index에는 빈도수가 높은 상위 5개의 단어만 저장되어있다.

저장한 word_to_index를 사용하여 sentences에서 첫번째 문장은 **단어 토큰화가 된 상태로 저장된 ['barber', 'person']**였다면, **각 단어를 정수인 [1, 5]로 인코딩**하는 작업을 진행할 것이다.
 


두번째 문장인 **['barber', 'good', 'person']**에는 더 이상 word_to_index에는 존재하지 않는 단어인 'good'이라는 단어가 있다.

이처럼 단어 집합에 존재하지 않는 단어들을 **Out-Of-Vocabulary(단어 집합에 없는 단어)의 약자로 'OOV'**라고 하는데, word_to_index에 'OOV'란 단어를 새롭게 추가하고, 단어 집합에 없는 단어들은 'OOV'의 인덱스로 인코딩하자.

In [10]:
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [11]:
word_to_index['OOV'] = len(word_to_index) + 1
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'OOV': 6}


이제 word_to_index를 사용하여 sentences의 모든 단어들을 맵핑되는 정수로 인코딩하자.

In [12]:
encoded = []
for s in sentences :
    temp = []
    for w in s :
        try :
            temp.append(word_to_index[w])
        except KeyError :
            temp.append(word_to_index['OOV'])
            
    encoded.append(temp)
print(encoded)

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


지금까지 파이썬의 dictionary 자료형으로 정수 인코딩을 진행했다. 그런데 이보다는 좀 더 쉽게 하기 위해서 Counter, FreqDist, enumerate 또는 케라스 토크나이저를 사용하는 것을 권장한다.

### 2) Counter 사용하기

In [13]:
from collections import Counter
print(sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


현재 sentences는 단어 토큰화가 된 결과가 저장되어져 있다.

단어 집합(vocabulary)을 만들기 위해서 sentences에서 문장의 경계인 [,]를 제거하고 단어들을 하나의 리스트로 만들자.

In [14]:
words = sum(sentences, [])
print(words)

# words = np.hstack(sentences)도 같은 결과

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


이를 파이썬의 **Counter()**의 입력으로 사용하면 중복을 제거하고 단어의 빈도수를 기록한다.

In [15]:
vocab = Counter(words)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


단어를 key로, 단어에 대한 빈도수가 값(value)로 저장된 것을 확인할 수 있다.

vocab에 단어를 입력하면 빈도수를 리턴한다.

In [16]:
# 'barber'라는 단어의 빈도수 출력
print(vocab['barber'])

8


barber라는 단어가 총 8번 등장했다. **most_common()**는 상위 빈도수를 가진 주어진 수의 단어만을 리턴한다. 등장 빈도수가 높은 단어들을 원하는 개수만큼만 얻을 수 있다. 등장 빈도수 상위 5개의 단어만 단어 집합으로 저장해보자.

In [17]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여한다.

In [18]:
word_to_index = {}

i = 0

for (word, frequency) in vocab :
    i = i+1
    word_to_index[word] = i
    
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### 3) NLTK의 FreqDist 사용하기

NLTK에서는 빈도수 계산 도구인 FreqDist()를 지원한다.


위에서 사용한 Counter()랑 같은 방법으로 사용할 수 있다.

In [19]:
from nltk import FreqDist
import numpy as np

In [20]:
# np.hstack으로 문장 경계를 제거하여 입력으로 사용
vocab = FreqDist(np.hstack(sentences))
vocab

FreqDist({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, ...})

In [21]:
# Counter와 마찬가지로 빈도수 출력
print(vocab['barber'])

8


**most_common()**으로 상위 빈도수를 가진 주어진 수의 단어만을 리턴

In [22]:
vocab_size = 5
vocab = vocab.most_common(vocab_size) # 등장 빈도수가 높은 상위 5개의 단어만 저장
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

앞서 Counter()를 사용했을 때와 결과가 같다. 마찬가지로 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여한다. 그러나 이번에는 **enumerate()**를 사용하여 좀 더 짧은 코드로 인덱스를 부여하고자 한다.

In [23]:
word_to_index = {word[0] : index + 1 for index, word in enumerate(vocab)} 

In [24]:
print(word_to_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### 4) enumerate 이해하기

enumerate()는 순서가 있는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아 인덱스를 순차적으로 함께 리턴한다는 특징이 있다. 

In [25]:
test=['a', 'b', 'c', 'd', 'e']
for index, value in enumerate(test) : # 입력의 순서대로 0부터 인덱스를 부여
    print('value  : {}, index : {}'.format(value,index))

value  : a, index : 0
value  : b, index : 1
value  : c, index : 2
value  : d, index : 3
value  : e, index : 4


위의 출력 결과는 리스트의 모든 토큰에 대해서 인덱스가 순차적으로 증가되며 부여된 것을 보여준다.